# Imports

In [1]:
import os

import pandas as pd
from ray.tune import Analysis

from rayTune_common.constants import metric, mode
from rayTune_common.test import test_model
from rayTune_common.utils import config_to_model

Analyse ray tune logs

In [2]:
path_to_run_results = "/home/knut/Documents/project/UnseededRun_results"
list_runs = [f.path for f in os.scandir(path_to_run_results) if f.is_dir()]

for path_to_run in list_runs:
    print(f'Run: {path_to_run}')
    data = []
    # path_to_run_results = "/home/knut/Documents/project/run_results/run6"
    path_to_csv = os.path.join(path_to_run, "results.csv")
    path_to_config_csv = os.path.join(path_to_run, "config.csv")

    list_experiments = [f.path for f in os.scandir(path_to_run) if f.is_dir()]
    list_experiments.sort(key=lambda x: x.split("_")[-1])

    for experient_number, path_to_experiment in enumerate(list_experiments):
        print(f"Run: {path_to_run} -- Experiment: {experient_number} -- {path_to_experiment}")
        experiment_data = {}

        best_trial_analysis = Analysis(path_to_experiment, default_metric=metric, default_mode=mode)
        best_trial_config = best_trial_analysis.get_best_config(metric=metric, mode=mode)
        best_trial_logdir = best_trial_analysis.get_best_logdir(metric=metric, mode=mode)
        list_best_trial_checkpoints = [f.path for f in os.scandir(best_trial_logdir) if f.is_dir()]
        list_best_trial_checkpoints.sort(key=lambda x: int(x.split("_")[-1]))
        best_trial_checkpoint_path = os.path.join(list_best_trial_checkpoints[-1], "checkpoint")
        best_trial_model = config_to_model(config=best_trial_config, checkpoint_path=best_trial_checkpoint_path)
        best_trial_mse = test_model(model=best_trial_model, batch_size=64)
        # print(f'mse: {best_trial_mse}, trial: {best_trial_logdir.split("_")[4]}')

        list_experiment_trials = [f.path for f in os.scandir(path_to_experiment) if f.is_dir()]
        list_experiment_trials.sort(key=lambda x: int(x.split("_")[4]))

        for trial_number, path_to_trial in enumerate(list_experiment_trials):
            list_trial_checkpoints = [f.path for f in os.scandir(path_to_trial) if f.is_dir()]
            list_trial_checkpoints.sort(key=lambda x: int(x.split("_")[-1]))

            trial_checkpoint_path = os.path.join(list_trial_checkpoints[-1], "checkpoint")

            trial_analysis = Analysis(path_to_trial, default_metric=metric, default_mode=mode)
            trial_config = trial_analysis.get_best_config(metric=metric, mode=mode)

            trial_model = config_to_model(config=trial_config, checkpoint_path=trial_checkpoint_path)

            trial_mse = test_model(model=trial_model, batch_size=64)
            experiment_data[trial_number] = trial_mse

        sorted_experiment_data = dict(sorted(experiment_data.items()))
        data.append(sorted_experiment_data)

    # Convert into pandas dataframe
    df = pd.DataFrame(data)

    # Find best trial based on test mse
    # Store the config of the best model as csv file along with test mse and trial number
    data = []
    column_names = 1
    col_index_of_min = df.idxmin(axis=1)
    value_of_min = df.min(axis=1)
    for i in range(len(col_index_of_min)):
        if (path_to_run[-3:-1] == "rs"):
            path = os.path.join(path_to_run, "rs_" + str(i).rjust(3, "0"))
        else:
            path = os.path.join(path_to_run, "xp_" + str(i).rjust(3, "0"))
        list_experiment_trials = [f.path for f in os.scandir(path) if f.is_dir()]
        list_experiment_trials.sort(key=lambda x: int(x.split("_")[4]))
        path_to_best_test_mse = list_experiment_trials[col_index_of_min[i]]

        list_trial_checkpoints = [f.path for f in os.scandir(path_to_best_test_mse) if f.is_dir()]
        list_trial_checkpoints.sort(key=lambda x: int(x.split("_")[-1]))
        trial_checkpoint_path = os.path.join(list_trial_checkpoints[-1], "checkpoint")
        trial_analysis = Analysis(path_to_best_test_mse, default_metric=metric, default_mode=mode)
        trial_config = trial_analysis.get_best_config(metric=metric, mode=mode)
        trial_model = config_to_model(config=trial_config, checkpoint_path=trial_checkpoint_path)
        trial_mse = test_model(model=trial_model, batch_size=64)
        assert (trial_mse == value_of_min[i])

        trial_config["mse"] = trial_mse
        trial_config["trial number"] = col_index_of_min[i]

        data.append(trial_config)

    config_df = pd.DataFrame(data)

    # Add mean and variance to test results
    df["best"] = df.min(axis=1)
    df.loc["mean"] = df.mean(axis=0)
    df.loc["var"] = df.var(axis=0)

    # Store dataframes as csv
    df.to_csv(path_to_csv)
    config_df.to_csv(path_to_config_csv)

Run: /home/knut/Documents/project/UnseededRun_results/run1
Run: /home/knut/Documents/project/UnseededRun_results/run1 -- Experiment: 0 -- /home/knut/Documents/project/UnseededRun_results/run1/xp_000
Run: /home/knut/Documents/project/UnseededRun_results/run1 -- Experiment: 1 -- /home/knut/Documents/project/UnseededRun_results/run1/xp_001
Run: /home/knut/Documents/project/UnseededRun_results/run1 -- Experiment: 2 -- /home/knut/Documents/project/UnseededRun_results/run1/xp_002
Run: /home/knut/Documents/project/UnseededRun_results/run1 -- Experiment: 3 -- /home/knut/Documents/project/UnseededRun_results/run1/xp_003
Run: /home/knut/Documents/project/UnseededRun_results/run1 -- Experiment: 4 -- /home/knut/Documents/project/UnseededRun_results/run1/xp_004
Run: /home/knut/Documents/project/UnseededRun_results/run1 -- Experiment: 5 -- /home/knut/Documents/project/UnseededRun_results/run1/xp_005
Run: /home/knut/Documents/project/UnseededRun_results/run1 -- Experiment: 6 -- /home/knut/Documents/p

Run: /home/knut/Documents/project/UnseededRun_results/run4 -- Experiment: 12 -- /home/knut/Documents/project/UnseededRun_results/run4/xp_012
Run: /home/knut/Documents/project/UnseededRun_results/run4 -- Experiment: 13 -- /home/knut/Documents/project/UnseededRun_results/run4/xp_013
Run: /home/knut/Documents/project/UnseededRun_results/run4 -- Experiment: 14 -- /home/knut/Documents/project/UnseededRun_results/run4/xp_014
Run: /home/knut/Documents/project/UnseededRun_results/plots
Run: /home/knut/Documents/project/UnseededRun_results/run5
Run: /home/knut/Documents/project/UnseededRun_results/run5 -- Experiment: 0 -- /home/knut/Documents/project/UnseededRun_results/run5/xp_000
Run: /home/knut/Documents/project/UnseededRun_results/run5 -- Experiment: 1 -- /home/knut/Documents/project/UnseededRun_results/run5/xp_001
Run: /home/knut/Documents/project/UnseededRun_results/run5 -- Experiment: 2 -- /home/knut/Documents/project/UnseededRun_results/run5/xp_002
Run: /home/knut/Documents/project/Unse

Run: /home/knut/Documents/project/UnseededRun_results/rs5 -- Experiment: 9 -- /home/knut/Documents/project/UnseededRun_results/rs5/rs_009
Run: /home/knut/Documents/project/UnseededRun_results/rs5 -- Experiment: 10 -- /home/knut/Documents/project/UnseededRun_results/rs5/rs_010
Run: /home/knut/Documents/project/UnseededRun_results/rs5 -- Experiment: 11 -- /home/knut/Documents/project/UnseededRun_results/rs5/rs_011
Run: /home/knut/Documents/project/UnseededRun_results/rs5 -- Experiment: 12 -- /home/knut/Documents/project/UnseededRun_results/rs5/rs_012
Run: /home/knut/Documents/project/UnseededRun_results/rs5 -- Experiment: 13 -- /home/knut/Documents/project/UnseededRun_results/rs5/rs_013
Run: /home/knut/Documents/project/UnseededRun_results/rs5 -- Experiment: 14 -- /home/knut/Documents/project/UnseededRun_results/rs5/rs_014
Run: /home/knut/Documents/project/UnseededRun_results/rs1
Run: /home/knut/Documents/project/UnseededRun_results/rs1 -- Experiment: 0 -- /home/knut/Documents/project/Un